# <span style="text-decoration: underline; color: #ffff00">March Tabular Playground Series competition</span> 
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTJ9fIwYofhSehmxC8QvHDXMU8AY9VP8DvcdA&usqp=CAU)
## <span style='color:#ffd700'> Importación de librerías </span>

In [1]:
import zipfile
import os
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

## <span style='color:#ffd700'> Descarga de información y generación de DataFrames </span>
Descargo la información directo desde la API de Kaggle, la descomprimo y borro el zip original.  
Si no se tiene instalado kaggle, descomentar la siguiente celda y correrla. Luego entrar a la consola y correr $kaggle$. Esto creará la carpeta .kaggle en C:\Users\user\, donde se debe copiar el arcivo token (.json) que se obtiene de la subsección API en Account dentro del usuario propio de kaggle.

In [2]:
# conda install -c conda-forge kaggle

In [3]:
file = pathlib.Path("datasets/2022_03/train.csv")
if file.exists():
    print('Files already exists')
else:
    print('Creating the file')
    !kaggle competitions files -c tabular-playground-series-mar-2022
    !kaggle competitions download -c tabular-playground-series-mar-2022
    with zipfile.ZipFile("tabular-playground-series-mar-2022.zip", "r") as zip_ref:
        zip_ref.extractall(r"C:\Users\alanp\Data Science\kaggle_tabular_playground\datasets\2022_03")
    os.remove("tabular-playground-series-mar-2022.zip")

Creating the file
name                   size  creationDate         
---------------------  ----  -------------------  
train.csv              30MB  2022-01-14 15:58:08  
sample_submission.csv  25KB  2022-01-14 15:58:08  
test.csv               78KB  2022-01-14 15:58:08  




  0%|          | 0.00/4.71M [00:00<?, ?B/s]
 21%|##1       | 1.00M/4.71M [00:00<00:00, 5.16MB/s]
 43%|####2     | 2.00M/4.71M [00:00<00:00, 6.90MB/s]
 64%|######3   | 3.00M/4.71M [00:00<00:00, 7.04MB/s]
 85%|########5 | 4.00M/4.71M [00:00<00:00, 7.71MB/s]
100%|##########| 4.71M/4.71M [00:00<00:00, 7.45MB/s]


Luego genero los DataFrame desde los .csv utilizando pandas

In [6]:
train = pd.read_csv("datasets/2022_03/train.csv", index_col='row_id')
test = pd.read_csv("datasets/2022_03/test.csv", index_col='row_id')

##### hola
hola
###### hola
hola

### Predicción del test con el modelo elegido

In [8]:
# Habilitar cuando se tenga el modelo entrenado
#pred_test = model.predict(test)

In [9]:
#predictions = la.inverse_transform(pred_test)
#predictions

In [10]:
# output = pd.DataFrame({'row_id': test.index, 'target': predictions})
# output.to_csv('datasets/2022_03/submission1.csv', index=False)
# print("Your submission was successfully saved!")

In [11]:
# !kaggle competitions submit -c tabular-playground-series-mar-2022 -f datasets/2022_03/submission.csv -m "1- "